In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
import pandas as pd, numpy as np
from tqdm.notebook import tqdm
import os, sys, pickle, glob, gc
from collections import Counter
#import cudf, 
import itertools
#print('We will use RAPIDS version',cudf.__version__)
import warnings
warnings.filterwarnings('ignore')

In [ ]:
INPUT_DIR = '/content/drive/MyDrive/kaggle/2022/OTTO/input/otto-chunk-data-inparquet-format/'
OUTPUT_DIR = '/content/drive/MyDrive/kaggle/2022/OTTO/output/EDA/'

# load

In [ ]:
%%time

files = glob.glob(INPUT_DIR+'train_parquet/*')
train_df = pd.DataFrame()

#full train load
for file in tqdm(files):
  tmp_train = pd.read_parquet(file)
  train_df = pd.concat([train_df, tmp_train])
train_df

train_df['date'] = pd.to_datetime(train_df['ts'], unit='ms')
train_df['day'] = train_df['date'].dt.strftime('%d')

#train_df.to_csv(OUTPUT_DIR+ 'train_df.csv',index=False)

  0%|          | 0/129 [00:00<?, ?it/s]

CPU times: user 18min 23s, sys: 1min 9s, total: 19min 32s
Wall time: 19min 31s


In [ ]:
'''
#csvにするとメモリたくさん使いそう
%%time
train_df = pd.read_csv(OUTPUT_DIR + 'train_df.csv')
train_df
'''

"\n%%time\ntrain_df = pd.read_csv(OUTPUT_DIR + 'train_df.csv')\ntrain_df\n"

#train all dayでtop20人気商品のセッションごとの割合見てみる

click, cart, order

In [ ]:
#全てのセッションにおけるtop20click,cart,orderの割合
all_popular_aids = list(train_df.loc[:,'aid'].value_counts().index.values[:20])
unique_session = train_df['session'].unique()
len(train_df.query(f'aid == {all_popular_aids}'))/len(unique_session)

0.13458036761715064

click

In [ ]:
#全clickセッションにおけるtop20clickの割合
click_popular_aids = list(train_df.loc[train_df['type']=='clicks','aid'].value_counts().index.values[:20])
unique_click_session = train_df.loc[train_df['type']=='clicks','session'].unique()
len(train_df.query(f'aid == {click_popular_aids}'))/len(unique_click_session)

0.13406749061359888

cart

In [ ]:
#全cartセッションにおけるtop20cartの割合
cart_popular_aids = list(train_df.loc[train_df['type']=='carts','aid'].value_counts().index.values[:20])
unique_cart_session = train_df.loc[train_df['type']=='carts','session'].unique()
len(train_df.query(f'aid == {click_popular_aids}'))/len(unique_cart_session)

0.45383742540096245

order

In [ ]:
#全orderセッションにおけるtop20cartの割合
order_popular_aids = list(train_df.loc[train_df['type']=='orders','aid'].value_counts().index.values[:20])
unique_order_session = train_df.loc[train_df['type']=='orders','session'].unique()
len(train_df.query(f'aid == {click_popular_aids}'))/len(unique_order_session)

1.0633958008728812

#train dayごとのtop20人気商品

#Load

all

In [ ]:
#load
with open(OUTPUT_DIR + 'train_days_click_cart_order_popular_aids.pkl','rb') as f:
  train_days_click_cart_order_popular_aids = pickle.load(f)

In [ ]:
#dayごとの人気商品top20を見てるdayのセッション割合
#例：7/31の人気商品を7/31のセッションの何割がclick,cart,orderしてるか。

train_df['day'] = train_df['day'].astype('int8')
train_days = [31, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28]

for day in tqdm(train_days):
  unique_sesion_day = train_df[train_df['day']==day]['session'].unique()
  popular_aids = train_days_click_cart_order_popular_aids[day]
  ratio = len(train_df.loc[train_df['day']==day,:].query(f'aid == {popular_aids}'))/len(unique_sesion_day)
  print(f'{day}day ratio :',ratio*100,'%')

  0%|          | 0/29 [00:00<?, ?it/s]

31day ratio : 6.836898864233716 %
1day ratio : 5.765689086558725 %
2day ratio : 6.194989844278944 %
3day ratio : 5.959358918250004 %
4day ratio : 13.292581445447688 %
5day ratio : 6.7033958777528655 %
6day ratio : 6.990725549092929 %
7day ratio : 9.307892485675824 %
8day ratio : 7.742464158520103 %
9day ratio : 7.008093213158935 %
10day ratio : 6.8958148955627365 %
11day ratio : 7.159027105320975 %
12day ratio : 6.391710397029332 %
13day ratio : 6.43974275750668 %
14day ratio : 6.387088289590446 %
15day ratio : 6.036328335260883 %
16day ratio : 6.437676981585677 %
17day ratio : 8.205598035178028 %
18day ratio : 6.678710183835412 %
19day ratio : 5.477907382877092 %
20day ratio : 5.882489883173838 %
21day ratio : 7.2241188917925125 %
22day ratio : 4.950730318757548 %
23day ratio : 21.1299279192306 %
24day ratio : 10.561381264542202 %
25day ratio : 7.649519530195656 %
26day ratio : 6.471682408465022 %
27day ratio : 6.959809432485289 %
28day ratio : 6.579109101652571 %


click

In [ ]:
#load
with open(OUTPUT_DIR + 'train_days_click_popular_aids.pkl','rb') as f:
  train_days_click_popular_aids = pickle.load(f)

In [ ]:
train_days = [31, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28]

types = 'clicks'

for day in tqdm(train_days):
  day_df = train_df[train_df['day']==day]
  unique_session_day = day_df[day_df['type']==types]['session'].unique()
  popular_aids = train_days_click_popular_aids[day]
  ratio = len(train_df.loc[train_df['day']==day,:].query(f'aid == {popular_aids}'))/len(unique_sesion_day)
  print(f'{day}day ratio :',ratio*100,'%')

  0%|          | 0/29 [00:00<?, ?it/s]

31day ratio : 0.3167710969698675 %
1day ratio : 5.13971841581871 %
2day ratio : 5.736977805290181 %
3day ratio : 5.805174668382572 %
4day ratio : 12.626119768797084 %
5day ratio : 5.780072891413711 %
6day ratio : 5.737125898369643 %
7day ratio : 8.417314450478415 %
8day ratio : 6.13423749094781 %
9day ratio : 6.418428110435971 %
10day ratio : 6.20554430870891 %
11day ratio : 6.496695302931799 %
12day ratio : 6.0975844537808905 %
13day ratio : 6.3041742996308034 %
14day ratio : 7.661965698680935 %
15day ratio : 6.534755224353611 %
16day ratio : 6.0620421147099375 %
17day ratio : 7.682328497107002 %
18day ratio : 5.975185523605296 %
19day ratio : 4.928537684505466 %
20day ratio : 5.39103237166624 %
21day ratio : 7.396434807205024 %
22day ratio : 4.442422151170454 %
23day ratio : 19.28593959875661 %
24day ratio : 9.35044894417039 %
25day ratio : 6.477443202601699 %
26day ratio : 5.5600806218724586 %
27day ratio : 6.0837377508511645 %
28day ratio : 6.558153930908654 %


cart

In [ ]:
#load
with open(OUTPUT_DIR + 'train_days_cart_popular_aids.pkl','rb') as f:
  train_days_cart_popular_aids = pickle.load(f)

In [ ]:
train_days = [31, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28]

types = 'carts'

for day in tqdm(train_days):
  day_df = train_df[train_df['day']==day]
  unique_session_day = day_df[day_df['type']==types]['session'].unique()
  popular_aids = train_days_cart_popular_aids[day]
  ratio = len(train_df.loc[train_df['day']==day,:].query(f'aid == {popular_aids}'))/len(unique_sesion_day)
  print(f'{day}day ratio :',ratio*100,'%')

  0%|          | 0/29 [00:00<?, ?it/s]

31day ratio : 0.24079934720570573 %
1day ratio : 4.37851998738247 %
2day ratio : 4.990366545180977 %
3day ratio : 4.9848130547011404 %
4day ratio : 11.083137973879342 %
5day ratio : 5.13712678692812 %
6day ratio : 5.231462078545608 %
7day ratio : 6.908468110376734 %
8day ratio : 5.459969700155942 %
9day ratio : 5.240569802932539 %
10day ratio : 5.222206261079213 %
11day ratio : 5.661006055526019 %
12day ratio : 5.198067089126858 %
13day ratio : 5.347418959764591 %
14day ratio : 5.989920785011796 %
15day ratio : 5.385108648487748 %
16day ratio : 4.902695442139294 %
17day ratio : 5.3934759074773675 %
18day ratio : 5.130832831050972 %
19day ratio : 4.25315919561763 %
20day ratio : 4.524317624113108 %
21day ratio : 6.410060851446351 %
22day ratio : 3.5965144812817753 %
23day ratio : 17.613894685087473 %
24day ratio : 7.220870461502464 %
25day ratio : 5.332091326040243 %
26day ratio : 4.501733429495106 %
27day ratio : 5.247382084587805 %
28day ratio : 4.477446164463288 %


order

In [ ]:
#load
with open(OUTPUT_DIR + 'train_days_order_popular_aids.pkl','rb') as f:
  train_days_order_popular_aids = pickle.load(f)

In [ ]:
train_days = [31, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28]

types = 'orders'

for day in tqdm(train_days):
  day_df = train_df[train_df['day']==day]
  unique_session_day = day_df[day_df['type']==types]['session'].unique()
  popular_aids = train_days_order_popular_aids[day]
  ratio = len(train_df.loc[train_df['day']==day,:].query(f'aid == {popular_aids}'))/len(unique_sesion_day)
  print(f'{day}day ratio :',ratio*100,'%')

  0%|          | 0/29 [00:00<?, ?it/s]

31day ratio : 0.1922988636818013 %
1day ratio : 3.890183057855524 %
2day ratio : 3.878853937276657 %
3day ratio : 3.6195429551381637 %
4day ratio : 4.984516868542216 %
5day ratio : 4.520393157507357 %
6day ratio : 4.415098978009659 %
7day ratio : 5.575408255596807 %
8day ratio : 4.388071991007788 %
9day ratio : 4.386591060213165 %
10day ratio : 4.313729265117712 %
11day ratio : 5.116689941962322 %
12day ratio : 4.584665553993996 %
13day ratio : 4.825464901199702 %
14day ratio : 5.269521999967419 %
15day ratio : 4.080186478805659 %
16day ratio : 2.9828908065297197 %
17day ratio : 4.640052365712897 %
18day ratio : 4.619763613826562 %
19day ratio : 3.078558935862368 %
20day ratio : 3.379854306028425 %
21day ratio : 5.922612480396179 %
22day ratio : 3.333427125616993 %
23day ratio : 3.84819866982796 %
24day ratio : 5.132461854925058 %
25day ratio : 4.804361637376324 %
26day ratio : 3.9833336048373122 %
27day ratio : 4.431241123671049 %
28day ratio : 3.5394986456887887 %


cart, orderで割合異なるけど、
orderはorder≒セッション数だから問題ない想定
でも8/23はorder低いけどどゆこと？